# UAS

https://colab.research.google.com/drive/1pVD0YcX0c1i8C-i7lXt20wHI8l2j1kHe?usp=sharing

*   **NLTK** nantinya digunakan untuk melakukan tokenisasi.
*   **Pandas** digunakan untuk mengelola, membaca dan menganalisis data tabular.
*   **CountVectorizer** untuk melakukan ekstraksi fitur dalam bentuk term frequency yang menggunakan library dari sklearn.
*   **Numpy** untuk array dan operasi matematika pada array.
*   **re** Ekspresi reguler digunakan untuk pencocokan pola string dan manipulasi teks.
*   **Train_test_split** untuk membagi data training dan data testing yang mana menggunakan library pada sklearn.
*   **Latent Dirichlet Allocation** untuk melakukan ekstrak data menjadi data dokumen per topik.
*   **GausianNB** untuk melatih data training sehingga dapat digunakan untuk mendeteksi data baru.

In [3]:
!pip install nltk

In [2]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
import nltk
import pickle
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.naive_bayes import GaussianNB

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Load Data

Dataset ini diperoleh dengan menggabungkan hasil crawling berita dari detik.com dan pojokbaca.id yang masing-masing mengambil kategori otomotif, politik dan sport/ olahraga.

In [4]:
# data dari hasil crawling detik
data1 = pd.read_csv('/content/drive/MyDrive/ppw/ppw/crawling_berita_detik.csv')
data1

,Judul,Isi,Kategori
0,Starling di Senayan Deklarasi Dukungan ke Prab...,Kumpulan para starling (starbucks keliling) me...,Politik
1,Profil 2 Moderator Pemandu Debat Perdana Capre...,Ardianto Wijaya dan Valerina Daniel merupakan ...,Politik
2,"Siapa yang Punya, Nih? Ratusan Motor Terlantar...",Area parkir sepeda motor di Bandara Ngurah Rai...,Otomotif
3,5 Mobil Ferrari Termahal yang Pernah Dijual Di...,Siapa yang tidak tahu Ferrari? Pabrikan otomot...,Otomotif
4,'Final' MotoGP 2023 Bagnaia Vs Martin: Ini Pre...,Francesco Bagnaia dan Jorge Martin hadapi 'fin...,Sport
...,...,...,...
875,Ganjar Ingin Pajak Diurus Lembaga Langsung di ...,"Calon presiden (Capres) nomor urut 3, Ganjar P...",Politik
876,"Richardo Kalahkan Bima, Maju ke 16 Besar BNI S...",Pebulutangkis muda Richardo Regino Mozes Munde...,Sport
877,PDIP Banten Tak Ambil Pusing soal Ada Kader Du...,"Ketua DPD PDIP Banten, Ade Sumardi, tak ingin ...",Politik
878,Mobil Kembaran Limusin Vladimir Putin Bakal Di...,Mobil kepresidenan Vladimir Putin versi sipil ...,Otomotif


In [5]:
# data dari hasil crawling pojok baca
data2 = pd.read_csv('/content/drive/MyDrive/ppw/ppw/crawling_berita_pb.csv')
data2

,Judul,Isi,Kategori
0,Jubir Nilai A Plus untuk Anies di Debat Capres...,"SURABAYA, POJOKBACA.ID -Debat Capresyang digel...",Politik
1,"Prediksi Filipina vs Timnas Indonesia, Laga Hi...",POJOKBACA.ID -Timnas Indonesiaakan menghadapi...,Sport
2,"Ternyata, 4 Area Tubuh Ini yang Disukai dan Di...","POJOKBACA.ID -Kucing, makhluk yang memikat ha...",Politik
3,Rumah Tangga Lionel Messi Retak Gara-gara Wart...,POJOKBACA.ID -PernikahanLionel MessidanAntone...,Sport
4,"Jika Prabowo-Gibran Terpilih di Pilpres 2024, ...","JAKARTA, POJOKBACA.ID -Calon presiden (capres...",Politik
...,...,...,...
595,Ganjar Janji Bakal Tindak Tegas Para Pengusaha...,"JAKARTA, POJOKBACA.ID -Calon presiden nomor ur...",Politik
596,"Tiba di Riau, TPD AMIN: Kedatangan Anies Baswe...","PEKANBARU, POJOKBACA.ID -Rabu (13/12/2023), a...",Politik
597,Dinamis dan Menggoda: Honda WR-V RS Versi The ...,"JAKARTA, POJOKBACA.ID -Ajang Gaikindo Indones...",Otomotif
598,"Dibanderol Rp40 Jutaan, Ini Spesifikasi dan Fi...","JAKARTA, POJOKBACA.ID -Honda, sebagai salah s...",Otomotif


In [6]:
# Penggabungan dataset dari crawling detik dan crawling pojok baca
combined_df = pd.concat([data1, data2]).reset_index(drop=True)
combined_df

,Judul,Isi,Kategori
0,Starling di Senayan Deklarasi Dukungan ke Prab...,Kumpulan para starling (starbucks keliling) me...,Politik
1,Profil 2 Moderator Pemandu Debat Perdana Capre...,Ardianto Wijaya dan Valerina Daniel merupakan ...,Politik
2,"Siapa yang Punya, Nih? Ratusan Motor Terlantar...",Area parkir sepeda motor di Bandara Ngurah Rai...,Otomotif
3,5 Mobil Ferrari Termahal yang Pernah Dijual Di...,Siapa yang tidak tahu Ferrari? Pabrikan otomot...,Otomotif
4,'Final' MotoGP 2023 Bagnaia Vs Martin: Ini Pre...,Francesco Bagnaia dan Jorge Martin hadapi 'fin...,Sport
...,...,...,...
1475,Ganjar Janji Bakal Tindak Tegas Para Pengusaha...,"JAKARTA, POJOKBACA.ID -Calon presiden nomor ur...",Politik
1476,"Tiba di Riau, TPD AMIN: Kedatangan Anies Baswe...","PEKANBARU, POJOKBACA.ID -Rabu (13/12/2023), a...",Politik
1477,Dinamis dan Menggoda: Honda WR-V RS Versi The ...,"JAKARTA, POJOKBACA.ID -Ajang Gaikindo Indones...",Otomotif
1478,"Dibanderol Rp40 Jutaan, Ini Spesifikasi dan Fi...","JAKARTA, POJOKBACA.ID -Honda, sebagai salah s...",Otomotif


In [7]:
# combined_df.to_csv('/content/drive/MyDrive/ppw/ppw/crawling_berita_gabungan.csv', index=False)

## Preprocessing

Cleaning adalah menghapus hal-hal yang tidak digunakan seperti tanda baca, angka, tag dan hastag. dalam proses cleaning ini digunakan library regex untuk mencari hal-hal yang diinginkan.

Casefolding adalah proses merubah huruf kapital atau besar menjadi huruf biasa atau kecil.

In [8]:
#Remove Puncutuation
clean_tag = re.compile('@\S+')
clean_url = re.compile('https?:\/\/.*[\r\n]*')
clean_hastag = re.compile('#\S+')
clean_symbol = re.compile('[^a-zA-Z]')
def clean_punct(text):
    text = clean_tag.sub('', str(text))
    text = clean_url.sub('', text)
    text = clean_hastag.sub(' ', text)
    text = clean_symbol.sub(' ', text)
    text = text.lower()
    return text
# Buat kolom tambahan untuk data description yang telah diremovepunctuation
preprocessing = combined_df['Isi'].apply(clean_punct)
clean=pd.DataFrame(preprocessing)
clean

,Isi
0,kumpulan para starling starbucks keliling me...
1,ardianto wijaya dan valerina daniel merupakan ...
2,area parkir sepeda motor di bandara ngurah rai...
3,siapa yang tidak tahu ferrari pabrikan otomot...
4,francesco bagnaia dan jorge martin hadapi fin...
...,...
1475,jakarta pojokbaca id calon presiden nomor ur...
1476,pekanbaru pojokbaca id rabu a...
1477,jakarta pojokbaca id ajang gaikindo indones...
1478,jakarta pojokbaca id honda sebagai salah s...


In [9]:
data_clean=[]
for i in range(len(preprocessing)):
  data_clean.append(preprocessing[i])

Tokenizing adalah memisahkan kalimat menjadi kata-kata berdasarkan spasi. dalam proses tokenizing dilakukan looping setiap datanya untuk dipisahkan menggunakan bantuan library nltk yang menghasilkan list setiap data yang sudah terpisah-pisah katanya.

In [10]:
tokenize=[]
for i in range(len(data_clean)):
  token=word_tokenize(data_clean[i])
  tokendata = []
  for x in token :
    tokendata.append(x)
  tokenize.append(tokendata)
  # print(tokendata)

Stopword adalah menghapus kata yang tidak penting dalam kalimat. hasil dari tokenize akan dilakukan pengecekan setiap kata dalam data untuk menghapus kata yang tidak perlu. Hasil pengecekan tadi kemudian dikembalikan dalam bentuk list.

In [11]:
stopword=[]
for i in range(len(tokenize)):
  listStopword =  set(stopwords.words('indonesian'))
  removed=[]
  for x in (tokenize[i]):
    if x not in listStopword:
       removed.append(x)
  stopword.append(removed)
  # print(removed)

In [12]:
join=[]
for i in range(len(stopword)):
  joinkata = ' '.join(stopword[i])
  join.append(joinkata)

hasilpreproses = pd.DataFrame(join, columns=['Isi'])
hasilpreproses

,Isi
0,kumpulan starling starbucks keliling mendeklar...
1,ardianto wijaya valerina daniel pembawa acara ...
2,area parkir sepeda motor bandara ngurah rai ba...
3,ferrari pabrikan otomotif italia dibilang jaja...
4,francesco bagnaia jorge martin hadapi final mo...
...,...
1475,jakarta pojokbaca id calon presiden nomor urut...
1476,pekanbaru pojokbaca id rabu atmosfer politik d...
1477,jakarta pojokbaca id ajang gaikindo indonesia ...
1478,jakarta pojokbaca id honda salah produsen sepe...


## Ekstraksi Fitur

Term frequency (TF) adalah proses menghitung frekuensi atau banyak kemunculan suatu kata dalam suatu dokumen.

$\textbf{TF} = \frac{Jumlah kemunculan kata}{Jumlah total kata dalam dokumen}$

dalam proses Term Frequency ini nantinya menggunakan salah satu library sklearn yaitu CountVectorizer

In [13]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(np.array(hasilpreproses['Isi']))

In [14]:
tf_name=vectorizer.get_feature_names_out()
tf_name

array(['aa', 'aalanyallamahmud', 'aan', ..., 'zulmi', 'zumba', 'zz'],
      dtype=object)

In [15]:
tf_array = X.toarray()
print(tf_array)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [16]:
df_tf= pd.DataFrame(tf_array, columns = tf_name)
df_tf

,aa,aalanyallamahmud,aan,aaron,ab,abadi,abadihonda,abah,abahnya,abaikan,...,zs,zubastian,zudan,zuhur,zul,zulhas,zulkifli,zulmi,zumba,zz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1476,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1477,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1478,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# pickle.dump(vectorizer,open('/content/drive/MyDrive/ppw/ppw/tf.sav','wb'))

In [18]:
# df_tf.to_csv('/content/drive/MyDrive/ppw/ppw/TF_UAS.csv', index=False)

## Split Data

Splitting Data merupakan proses pembagian dataset awal menjadi 2 data yaitu data training dan data testing. Dalam splitting data ini nantinya dibagi dengan perbandingan data training dan data testing yaitu 80:20

In [19]:
y = combined_df['Kategori']

In [20]:
X_train,X_test,y_train,y_test = train_test_split(df_tf,y,test_size=0.2,random_state=42)

## LDA + Modelling

*   Buat list kosong dengan nama datalda dan X_test_lda untuk menampung hasil topik per dokumen berdasarkan looping jumlah topik yng diinginkan mulai 3-51 topik
*   jalankan library lda dan lakukan fit transform pada data yang sudah di load sebelumnya. hasil lda disimpan ke dalam variabel datalda. proses ini berlangsung sampai range mencapai batas akhir.

In [21]:
datalda=[]
X_test_lda=[]
for i in range (3,51):
  lda = LatentDirichletAllocation(n_components=i, doc_topic_prior=0.2, topic_word_prior=0.1,random_state=42,max_iter=1)
  lda_top=lda.fit_transform(X_train)
  test_lda = lda.transform(X_test)
  datalda.append(lda_top)
  X_test_lda.append(test_lda)

*  Hasil LDA yang ditampung di dalam variabel datalda digunakan untuk melatih model dengan menggunakan metode Naive Bayes. dan data X_test_lda untuk menengetes model yang sudah dilatih dan hasil akurasi disimpan dalam variabel akurasi.
*   Selanjutnya membuat looping untuk menampilkan akurasi berdasarkan jumlah topiknya.


In [22]:
akurasi=[]
for i in range(len(datalda)):
  NB = GaussianNB()
  gaus=NB.fit(datalda[i], y_train)
  nlakurasi=gaus.score(X_test_lda[i], y_test)
  akurasi.append(nlakurasi)

In [23]:
for i in range(len(akurasi)):
  print ("topik", i+3, ":", akurasi[i])

topik 3 : 0.7364864864864865
topik 4 : 0.7195945945945946
topik 5 : 0.7128378378378378
topik 6 : 0.668918918918919
topik 7 : 0.6452702702702703
topik 8 : 0.7162162162162162
topik 9 : 0.7736486486486487
topik 10 : 0.8412162162162162
topik 11 : 0.8344594594594594
topik 12 : 0.793918918918919
topik 13 : 0.8141891891891891
topik 14 : 0.8277027027027027
topik 15 : 0.8378378378378378
topik 16 : 0.8243243243243243
topik 17 : 0.8141891891891891
topik 18 : 0.7533783783783784
topik 19 : 0.8175675675675675
topik 20 : 0.8040540540540541
topik 21 : 0.7804054054054054
topik 22 : 0.7871621621621622
topik 23 : 0.7162162162162162
topik 24 : 0.793918918918919
topik 25 : 0.7871621621621622
topik 26 : 0.6756756756756757
topik 27 : 0.793918918918919
topik 28 : 0.6959459459459459
topik 29 : 0.706081081081081
topik 30 : 0.7162162162162162
topik 31 : 0.7297297297297297
topik 32 : 0.7804054054054054
topik 33 : 0.6452702702702703
topik 34 : 0.7263513513513513
topik 35 : 0.7297297297297297
topik 36 : 0.773648648

**Dari percobaan dala mencoba menerapkan data training pada pembagian topik 1 hingga 50 yang mana didapatkan hasil terbaik pada jumlah topik sebanyak 10 dengan nilai akurasi 84,12%**

### Download model yang memiliki nilai akurasi terbaik

Dari model dengan jumlah topik yang memperoleh nilai akurasi terbaik akan disimpan dalam bentuk pickle untuk diimplementasikan dalam bentuk streamlit.

In [24]:
lda = LatentDirichletAllocation(n_components=10, doc_topic_prior=0.2, topic_word_prior=0.1,random_state=42,max_iter=1)
lda_top=lda.fit_transform(X_train)

In [25]:
NB = GaussianNB()
gaus=NB.fit(lda_top, y_train)

In [26]:
test_lda = lda.transform(X_test)
gaus.score(test_lda, y_test)

0.8412162162162162

In [27]:
# pickle.dump(lda,open('/content/drive/MyDrive/ppw/ppw/lda.sav','wb'))
# pickle.dump(NB,open('/content/drive/MyDrive/ppw/ppw/naive.sav','wb'))